In [75]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=.7)

In [78]:
# Simple sequential chain. Can take only one input variable while invoking the chain

from langchain.chains import SimpleSequentialChain

movie_suggestion_prompt = """You are movie magzine editor. Suggest a good {genre} movie to watch with my wife. Return just the name of the movie, nothing else."""
movie_suggestion_prompt_template = PromptTemplate(input_variables=['genre'], template=movie_suggestion_prompt)
movie_suggestion_chain = LLMChain(llm=llm, prompt=movie_suggestion_prompt_template)

movie_review_prompt = """You are movie magzine cirtic. Given the movie {movie}, provide a brief review of the movie"""
movie_review_prompt_template = PromptTemplate(input_variables=['movie'], template=movie_review_prompt)
movie_review_chain = LLMChain(llm=llm, prompt=movie_review_prompt_template)

simple_chain = SimpleSequentialChain(chains=[movie_suggestion_chain, movie_review_chain], verbose=True)
simple_chain.run('Comedy')



> Entering new SimpleSequentialChain chain...


Bridesmaids


Bridesmaids is a hilarious and heartwarming comedy that follows a group of bridesmaids on a wild and unpredictable journey as they attempt to support their best friend on her journey to the altar. Featuring an all-star cast including Kristen Wiig, Maya Rudolph, Melissa McCarthy, and Rose Byrne, the movie is a riot of laughs and emotion, with each actress bringing her own unique brand of comedy to the screen. The script is sharp and smart, and the characters are endearing and relatable, making for a delightful and unforgettable experience. Bridesmaids is a must-see for fans of comedy.

> Finished chain.


'\n\nBridesmaids is a hilarious and heartwarming comedy that follows a group of bridesmaids on a wild and unpredictable journey as they attempt to support their best friend on her journey to the altar. Featuring an all-star cast including Kristen Wiig, Maya Rudolph, Melissa McCarthy, and Rose Byrne, the movie is a riot of laughs and emotion, with each actress bringing her own unique brand of comedy to the screen. The script is sharp and smart, and the characters are endearing and relatable, making for a delightful and unforgettable experience. Bridesmaids is a must-see for fans of comedy.'

In [80]:
# Sequential chain. Can take multiple input variables while invoking hte chain

from langchain.chains import SequentialChain

movie_suggestion_prompt_v2 = """You are movie magzine editor. Suggest a good {genre} movie to watch with my {audience} during {occassion}. Return just the name of the movie, nothing else."""
movie_suggestion_prompt_template_v2 = PromptTemplate(input_variables=['genre', 'audience', 'occassion'], template=movie_suggestion_prompt_v2)
movie_suggestion_chain_v2 = LLMChain(llm=llm, prompt=movie_suggestion_prompt_template_v2, output_key='movie')

movie_review_chain_v2 = LLMChain(llm=llm, prompt=movie_review_prompt_template, output_key='review')

complex_chain = SequentialChain(chains=[movie_suggestion_chain_v2, movie_review_chain_v2], input_variables=['genre', 'audience', 'occassion'], output_variables=['movie', 'review'], verbose=True)
complex_chain({'genre':'Comedy', 'audience':'wife', 'occassion':'marraige anniversary'})



> Entering new SequentialChain chain...

> Finished chain.


{'genre': 'Comedy',
 'audience': 'wife',
 'occassion': 'marraige anniversary',
 'movie': '\n\nThe Proposal',
 'review': "\n\nThe Proposal is a charming romantic comedy starring Sandra Bullock and Ryan Reynolds. Bullock plays Margaret Tate, a successful but stressed-out executive who is in danger of being deported from the U.S. She proposes marriage to her assistant, Andrew Paxton (Reynolds), to save her from deportation. The two must travel to Alaska to meet Margaret's family, and the story follows their misadventures as they attempt to convince Margaret's family that their marriage is real. The Proposal is a lighthearted and entertaining movie that is sure to please fans of romantic comedies. The chemistry between Bullock and Reynolds is palpable, and the supporting cast of characters adds a touch of humor to the story. All in all, The Proposal is an enjoyable film with plenty of laughs."}

In [84]:
# Sequential chain with memory

from langchain.memory import SimpleMemory

movie_recommendation_prompt = """You are responsible for recommending a movie for {event}. Decide if the {movie} would be a good recommendation or not. Explain your reasoning briefly."""
movie_recommendation_prompt_template = PromptTemplate(input_variables=['event', 'movie'], template=movie_recommendation_prompt)
movie_recommendation_chain = LLMChain(llm=llm, prompt=movie_recommendation_prompt_template, output_key='recommendation')

complex_chain_with_memory = SequentialChain(
    chains=[movie_suggestion_chain_v2, movie_review_chain_v2, movie_recommendation_chain], 
    memory = SimpleMemory(memories={'event': 'thanksgiving'}),
    input_variables=['genre', 'audience', 'occassion'], 
    output_variables=['movie', 'review', 'recommendation'], 
    verbose=True)
complex_chain_with_memory({'genre':'Comedy', 'audience':'wife', 'occassion':'marraige anniversary'})



> Entering new SequentialChain chain...

> Finished chain.


{'genre': 'Comedy',
 'audience': 'wife',
 'occassion': 'marraige anniversary',
 'event': 'thanksgiving',
 'movie': '\n\nThe Hangover',
 'review': '\n\nThe Hangover is a hilarious comedy that follows a group of friends on a wild night in Las Vegas. Four friends, Phil, Stu, Alan, and Doug, set out for a wild bachelor party weekend in Vegas. What follows is a wild and unpredictable night of partying, misadventures, and non-stop laughs. With an excellent cast and a clever script, The Hangover delivers an unforgettable experience that will keep you laughing from start to finish. It is a must-see for any fan of comedy.',
 'recommendation': '\n\nThe Hangover is not a suitable recommendation for Thanksgiving. This movie is a comedy about a group of friends attempting to piece together the events of their wild night in Las Vegas after one of them gets married. It is filled with adult humor and language, which makes it inappropriate for a family gathering.'}